 # **Fine-tuning Wav2Vec2 for Torgo DataSet with 🤗 Transformers**

# **Ensure that GPU and RAM is set up: will be needed for training purpose**

In [1]:
speaker = "M02"
repo_name = "torgo_xlsr_finetune-{}-2".format(speaker)


In [2]:

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jul 24 22:21:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# ensure enough memory present so that training does not stop
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# **Packages needed:** <br>
`datasets`: to transform the dataset <br>
`transformers`: upgraded version of RNN (allows to process a large quantity of text) <br>
`librosa`: needed for the audio files <br>
`jiwer`: **most important:** WER metric

In [4]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.26.1
!pip install jiwer
!pip install librosa
!pip install huggingface-hub
# %cd /content/espnet/tools
!make CUDA_VERSION=10.2

# **Download the torgo dataset that will be finetuned against the state of the art model**

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
%cd /content
#!gdown 1TaP_UX2QiiKR_qWMeYtlMgHaXlYRliJ9

!mkdir downloads
%cd downloads
# Original Link
# !gdown 1Olr1d6Ro9gb5r9wF7JOk9YSmmbW97djs
# !tar -xzf torgo.tar.gz && ls torgo
# !ls torgo


/content
/content/downloads


In [7]:
!pwd

/content/downloads


In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
!pwd

/content/downloads


In [19]:
# copy Torgo dataset from Google Drive
!pwd
!cp -r /content/drive/MyDrive/Torgo /content/downloads/Torgo


/content/downloads


In [53]:
!cd ..

In [54]:
!pwd

/content/downloads


In [6]:
%cd /content

/content


In [7]:
!pwd

/content


In [8]:
# download the output.csv file
!cp -r /content/drive/MyDrive/output_modified.csv /content/

cp: cannot stat '/content/drive/MyDrive/output_modified.csv': No such file or directory


# **Connect to Hugging Face to store the results of the model**

In [83]:
# to store the model checkpoints, we will need to install another package
%%capture
!apt install git-lfs

# **Crucial stage: Preparation of data, Tokenizer and Feature Extractor**

ASR models transcribe speech to text which leads to the requirement of a feature extractor and tokenizer: <br>
`feature extractor`: processes speech signal to the required input format: audio processing: feature vector <br>
`tokenizer`: converts the model's output to text format <br>
`wave2vec2` has the following tokenizer: `wave2vec2CTCTokenizer` and feature extractor: `wave2vec2FeatureExtractor`

# **Tokenizer**

In [9]:
# load the dataset, observe structure, divide into training and test set (evaluation later)
from datasets import load_dataset, load_metric, DatasetDict, Dataset, Audio

data = load_dataset('csv', data_files='/content/output_modified.csv')
print(data)

FileNotFoundError: ignored

In [85]:
repo_name

'torgo_xlsr_finetune-M02-2'

In [86]:
# creating a train and testing dataset
torgo_dataset = DatasetDict()

torgo_dataset['train'] = data['train'].filter(lambda x: x != speaker, input_columns=['speaker_id'])
torgo_dataset['test'] = data['train'].filter(lambda x: x == speaker, input_columns=['speaker_id'])

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [87]:
# remove columns that we do not need
torgo_dataset = torgo_dataset.remove_columns(["Unnamed: 0"])
torgo_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'session', 'text', 'audio', 'speaker_id', 'duration'],
        num_rows: 4663
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'session', 'text', 'audio', 'speaker_id', 'duration'],
        num_rows: 764
    })
})

In [88]:
# ignore special characters: with no language model hard to classify them
# also convert all the text into lowercase: makes life much more easier
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [89]:
# use map function to carry out the process/transformation
torgo_dataset = torgo_dataset.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [90]:
# write a function that will first concatenate all the transcriptions to one single transcription and them we map them to characters
# In short: creating tokens: determine the length of array

def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = torgo_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=torgo_dataset.column_names["train"])


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [91]:
# we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into
# an enumerated dictionary

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'p': 0,
 'a': 1,
 'r': 2,
 'o': 3,
 'q': 4,
 'g': 5,
 'm': 6,
 'b': 7,
 'y': 8,
 'i': 9,
 'd': 10,
 'n': 11,
 "'": 12,
 'h': 13,
 't': 14,
 'l': 15,
 's': 16,
 'c': 17,
 'j': 18,
 'x': 19,
 'z': 20,
 'e': 21,
 'k': 22,
 ' ': 23,
 'u': 24,
 'w': 25,
 'f': 26,
 'v': 27}

In [92]:
# from the above tokens: we given the space token visibility by using the symbol (|)
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [93]:
# adding tokens for anything unknown discovered and padding for the blank token
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


Linear layer we add on top of the pretrained checkpoint will have an output dimension of 32.

In [94]:
# jsonify the file next
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [95]:
# instantiate an object of the tokenizer class
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [96]:
# upload tokenizer to the Hugging Face Repo
# repo_name = "base-torgo"

In [97]:
# push it to Hugging face to use it later
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/monideep2255/torgo_xlsr_finetune-M02-2/commit/18455693f0308c0c1c84479b96d95b10623682ef', commit_message='Upload tokenizer', commit_description='', oid='18455693f0308c0c1c84479b96d95b10623682ef', pr_url=None, pr_revision=None, pr_num=None)

# **Feature Extractor**

To convert speech to text: it has to first discretized: create individual units: called **sampling**

A higher sampling rate leads to a better approximation of the real speech signal but also necessitates more values per second

A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`.

In [98]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [99]:
# wrap the feature extractor and tokenizer into a single processor class: when testing will only need model and processor object
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

#**Prepare Dataset**

In [100]:
torgo_dataset = torgo_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [101]:
torgo_dataset["train"][4]["audio"]

{'path': '/content/downloads/Torgo/F01/Session1/wav_arrayMic/0012.wav',
 'array': array([-0.00201416, -0.00595093, -0.00567627, ...,  0.0005188 ,
         0.00094604, -0.00186157], dtype=float32),
 'sampling_rate': 16000}

In [102]:
# testing out sample audio files that have been loaded
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(torgo_dataset["train"]))

#print(torgo_dataset["train"][rand_int]["text"])
#ipd.Audio(data=np.asarray(torgo_dataset["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

# **Processing the dataset expected by the model**

1. load and resample the audio data: call batch["audio"]
2. extract values from the loaded audio file
3. encode the transcriptions to label ids

In [103]:
def prepare_dataset(batch):
    # load the the audio data into batch
    audio = batch["audio"]

    # extract the values from the audio files
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    # encode it to the label ids
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [104]:
torgo_dataset = torgo_dataset.map(prepare_dataset, remove_columns=torgo_dataset.column_names["train"], num_proc=4)

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Long input sequences require a lot of memory. Since `Wav2Vec2` is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences.

In [105]:
torgo_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 4663
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 764
    })
})

### changed

In [106]:
max_input_length_in_sec = 9.0
min_input_length_in_sec= 1.0
torgo_dataset["train"] = torgo_dataset["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
torgo_dataset["train"] = torgo_dataset["train"].filter(lambda x: x > min_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
torgo_dataset["test"] = torgo_dataset["test"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
torgo_dataset["test"] = torgo_dataset["test"].filter(lambda x: x > min_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])


  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [107]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


# **Training and Evaluation**

**Need for a  data collabtor** <br>
wave2vec2 has a much larger input length as compared to the output length. For the input size, it is efficient to pad training batches to the longest sample in the batch (not overall sample)

In [108]:
# data collator

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [109]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

In [110]:
# load the word error rate metric
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [111]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    print(pred_str)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### **Model assigning**

In [112]:
# assign the model
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    # "yip-i/uaspeech-pretrained",
    "facebook/wav2vec2-large-xlsr-53",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)


# model: yongjian/wav2vec2-large-a contains [self-training] and has best wer of 0.557

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.codevectors', 'project_hid.weight', 'project_q.bias', 'project_hid.bias', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

The first component of wav2vec2 has acoustic features from the raw speech signal. This portion has been pretrained sufficiently and does not need to be pretrained anymore and hence freezed.

# **Define the parameters that are related to model training**

To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

In [113]:
model.freeze_feature_encoder()

In [114]:
for name, param in model.named_parameters():
     print(name, param.requires_grad)

wav2vec2.masked_spec_embed True
wav2vec2.feature_extractor.conv_layers.0.conv.weight False
wav2vec2.feature_extractor.conv_layers.0.conv.bias False
wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight False
wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias False
wav2vec2.feature_extractor.conv_layers.1.conv.weight False
wav2vec2.feature_extractor.conv_layers.1.conv.bias False
wav2vec2.feature_extractor.conv_layers.1.layer_norm.weight False
wav2vec2.feature_extractor.conv_layers.1.layer_norm.bias False
wav2vec2.feature_extractor.conv_layers.2.conv.weight False
wav2vec2.feature_extractor.conv_layers.2.conv.bias False
wav2vec2.feature_extractor.conv_layers.2.layer_norm.weight False
wav2vec2.feature_extractor.conv_layers.2.layer_norm.bias False
wav2vec2.feature_extractor.conv_layers.3.conv.weight False
wav2vec2.feature_extractor.conv_layers.3.conv.bias False
wav2vec2.feature_extractor.conv_layers.3.layer_norm.weight False
wav2vec2.feature_extractor.conv_layers.3.layer_norm.bia

In [115]:
# clear out cuda memory
import torch
torch.cuda.empty_cache()

### changed

In [116]:
# parameters for the training
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir = repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=False,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [117]:
# pass all instances to the trainer as the final step before training
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=torgo_dataset["train"],
    eval_dataset=torgo_dataset["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4416
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16560
  Number of trainable parameters = 311261344
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will 

Step,Training Loss,Validation Loss,Wer
500,23.219400,3.297529,0.981969
1000,3.385600,3.222471,0.981969
1500,2.940300,2.780463,0.981969
2000,2.625500,2.382952,0.983356
2500,1.890100,1.792012,1.377947
3000,1.259400,1.784516,1.334258
3500,1.000800,1.692319,1.314840
4000,0.789600,1.544352,1.282247
4500,0.637300,1.554703,1.266990
5000,0.563900,1.592442,1.193481


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 714
  Batch size = 8


['i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i',

Saving model checkpoint to torgo_xlsr_finetune-M02-2/checkpoint-500
Configuration saved in torgo_xlsr_finetune-M02-2/checkpoint-500/config.json
Model weights saved in torgo_xlsr_finetune-M02-2/checkpoint-500/pytorch_model.bin
Feature extractor saved in torgo_xlsr_finetune-M02-2/checkpoint-500/preprocessor_config.json
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 714
  Batch size = 8


...

['grow   i', 'feed   i', 'alpha  i', 'pot  i', 'farm  i', 'sip i', 'dagger i', 'rave i', 'fair i', 'sheet i', 'ship i', 'park i', 'goadfathersligkhts to be my dern answers lankgage i', 'spurgble i', 'bit  i', 'will she yis snearly notcis sit yorso i', 'trouble i', 'chair i', 'raid i', 'thaugh  i', 'goadget i', 'barn i', 'torn i', 'go i', 'swarm i', 'share i', 'ig a tarn i', 'i can meat i', 'sheate  i', 'starp     i', 'rought  i', 'no    i', 'tair i', 'know i', 'yes i', 'store i', 'chair i', 'swore i', 'bat  i', 'stubble i', 'trop      i', 'she had your doksouit in gored swash fwater all year i', 'horn  i', 'sheet     i', 'paca i', 'slect  i', 'spaine    i', 'car    i', 'knoying  i', 'know      i', 'yeet the stel think sefs swoasly as ever i', 'musually minus several buttons i', 'flor   i', 'traine  i', 'bhotp   i', 'we have often uszged jumpd o more and smoe gleas i', 'theue jugs smuna left i', 'boarn i', 'beiaht  i', 'nowm  i', 'right  i', 'range   i', 'sip   i', 'stick    i', 'storm 

Saving model checkpoint to torgo_xlsr_finetune-M02-2/checkpoint-16000
Configuration saved in torgo_xlsr_finetune-M02-2/checkpoint-16000/config.json
Model weights saved in torgo_xlsr_finetune-M02-2/checkpoint-16000/pytorch_model.bin
Feature extractor saved in torgo_xlsr_finetune-M02-2/checkpoint-16000/preprocessor_config.json
Deleting older checkpoint [torgo_xlsr_finetune-M02-2/checkpoint-15000] due to args.save_total_limit
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  

['grow   i', 'feed   i', 'alpha  i', 'pot  i', 'farm  i', 'sip i', 'daggeor i', 'rave i', 'fair i', 'sheet i', 'ship i', 'park i', 'gadfathers ligkhts to beae my dernanswers lankgage i', 'spurbble i', 'bit  i', 'will she yis snearly notcis it yors o i', 'trouble i', 'chair i', 'raid i', 'thaugh  i', 'goadget i', 'barn i', 'torn i', 'go i', 'swarm i', 'share i', 'ig a tarn i', 'i can meat i', 'sheate  i', 'starp    i', 'rought    i', 'no    i', 'tair i', 'know i', 'yes i', 'store i', 'chair i', 'swore i', 'bat  i', 'stubble i', 'trop       i', 'she had your doksouit in gored swash fwater all year i', 'horn  i', 'sheet       i', 'paca i', 'sect  i', 'spaine    i', 'car      i', 'knoying  i', 'know     i', 'yeet the stel think sefs swoasly as ever i', 'musually minus several buttons i', 'flor  i', 'train  i', 'bhop   i', 'we have often uszged jumpd o more and smowe glas i', 'theu jugs smuna left i', 'born i', 'beat  i', 'nowm   i', 'right  i', 'range   i', 'sip   i', 'stick    i', 'storm 

Saving model checkpoint to torgo_xlsr_finetune-M02-2/checkpoint-16500
Configuration saved in torgo_xlsr_finetune-M02-2/checkpoint-16500/config.json
Model weights saved in torgo_xlsr_finetune-M02-2/checkpoint-16500/pytorch_model.bin
Feature extractor saved in torgo_xlsr_finetune-M02-2/checkpoint-16500/preprocessor_config.json
Deleting older checkpoint [torgo_xlsr_finetune-M02-2/checkpoint-15500] due to args.save_total_limit
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=16560, training_loss=1.3274605709573497, metrics={'train_runtime': 24071.3345, 'train_samples_per_second': 5.504, 'train_steps_per_second': 0.688, 'total_flos': 1.2378565275038376e+19, 'train_loss': 1.3274605709573497, 'epoch': 30.0})

In [ ]:
# push to trained model to huggingface
# trainer.push_to_hub("yip-i/" + repo_name)
trainer.push_to_hub(repo_name)

Cloning https://huggingface.co/yip-i/torgo_xlsr_finetune-M02-2 into local empty directory.
Saving model checkpoint to torgo_xlsr_finetune-M02-2
Configuration saved in torgo_xlsr_finetune-M02-2/config.json
Model weights saved in torgo_xlsr_finetune-M02-2/pytorch_model.bin
Feature extractor saved in torgo_xlsr_finetune-M02-2/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 32.0k/1.18G [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/yip-i/torgo_xlsr_finetune-M02-2
   5d18e70..ad97514  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/yip-i/torgo_xlsr_finetune-M02-2
   5d18e70..ad97514  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Automatic Speech Recognition', 'type': 'automatic-speech-recognition'}, 'metrics': [{'name': 'Wer', 'type': 'wer', 'value': 1.0790568654646324}]}
To https://huggingface.co/yip-i/torgo_xlsr_finetune-M02-2
   ad97514..613a746  main -> main

   ad97514..613a746  main -> main



'https://huggingface.co/yip-i/torgo_xlsr_finetune-M02-2/commit/ad97514361362b3d5d9ea484568eaca59b9fe17e'

# **Evaluate the model and generate WER**

In [ ]:
!pip install evaluate
!pip install jiwer

In [ ]:
import re
from datasets import load_dataset, DatasetDict, Dataset, Audio
from huggingface_hub import Repository
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from tqdm import tqdm
from evaluate import load

In [ ]:
# Already present in the training section

'''
def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        # batch["labels"] = processor(batch["text"]).input_ids
        batch["labels"] = batch["text"]
        # print(processor(batch["text"]))
    return batch
'''

In [ ]:
# Function to be used for prediction
def map_to_result(batch):
    with torch.no_grad():
      input_values = torch.tensor(batch["input_values"], device="cpu").unsqueeze(0)
      logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = batch["labels"]

    return batch

In [ ]:
def get_result(torgo_dataset):
    pred_str = []
    actual = []
    for i in range(torgo_dataset.num_rows):
      inputs = processor(torgo_dataset[i]["input_values"], sampling_rate=16_000, return_tensors="pt")
      with torch.no_grad():
        logits = model(**inputs).logits
      transcription = processor.batch_decode(logits.numpy()).text
      pred_str.append(transcription[0].lower())
      actual = processor.decode(torgo_dataset[i]["labels"]).text

    return pred_str, actual

In [ ]:
# change speaker and model_name (this is the model that was trained and should be on Huggingface)
target_lang="en"  # change to your target lang
speaker = "M02"
model_name = "yip-i/torgo_xlsr_finetune-" + speaker + "-2"

In [ ]:
"""Cloning and uploading of modeling files can be done conveniently with the `huggingface_hub`'s `Repository` class.

More information on how to use the `huggingface_hub` to upload any files, please take a look at the [official docs](https://huggingface.co/docs/hub/how-to-upstream).
"""
repo = Repository(local_dir="model_staging", clone_from=model_name)

In [ ]:
"""### Processing Data"""


data = load_dataset('csv', data_files='/content/output_modified.csv')
data = data.cast_column("audio", Audio(sampling_rate=16_000))
timit = data['train'].filter(lambda x: x == speaker, input_columns=['speaker_id'])


processor = Wav2Vec2Processor.from_pretrained("model_staging")
model = Wav2Vec2ForCTC.from_pretrained(model_name)

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'



timit = timit.map(remove_special_characters)


timit = timit.map(prepare_dataset, remove_columns=timit.column_names, num_proc=4)
timit = timit.filter(lambda x: x < 25 * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

timit[0]

In [ ]:
# Generate the WER
pred_str = []
actual = []
for i in tqdm(range(timit.num_rows)):
  inputs = processor(timit[i]["input_values"], sampling_rate=16_000, return_tensors="pt")
  with torch.no_grad():
    logits = model(**inputs).logits
  transcription = processor.batch_decode(logits.numpy()).text
  pred_str.append(transcription[0].lower())

  actual.append(timit[i]["labels"])

wer_metric = load("wer")
wer = wer_metric.compute(predictions = pred_str, references = actual)
print("WER LOCATION")
print(wer)